<a href="https://colab.research.google.com/github/hsunsmile/ComfyUI_VLM_nodes/blob/main/nightly/comic_diffusion_v2_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

!apt -y update -qq
!apt -y install -qq aria2

In [ ]:
!pip install -q protobuf==3.20.3 numpy==1.23.5
!pip install -q torch==2.8.0 torchvision torchaudio torchtext torchdata --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers triton -U
!pip install -q mediapipe addict yapf fvcore omegaconf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# #@title Environment Setup

from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"
os.environ['COMFYUI_PATH'] = WORKSPACE

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies


In [ ]:
%cd /content/drive/MyDrive
!git clone -b v1.2 https://github.com/camenduru/ComfyUI
%cd ComfyUI
!pip install -q -r requirements.txt
!git reset --hard
!git clone -b v1.2 https://github.com/camenduru/comfy_controlnet_preprocessors /content/drive/MyDrive/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
%cd /content/drive/MyDrive/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts
%cd /content/drive/MyDrive/ComfyUI

controlnet_dir = "/content/drive/MyDrive/ComfyUI/models/controlnet"
upscale_dir = "/content/drive/MyDrive/ComfyUI/models/upscale_models"
checkpoints_dir = "/content/drive/MyDrive/ComfyUI/models/checkpoints"

os.makedirs(controlnet_dir, exist_ok=True)
os.makedirs(upscale_dir, exist_ok=True)
os.makedirs(checkpoints_dir, exist_ok=True)

controlnet_files = [
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors", "control_v11e_sd15_ip2p_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors", "control_v11e_sd15_shuffle_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors", "control_v11p_sd15_canny_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors", "control_v11f1p_sd15_depth_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors", "control_v11p_sd15_inpaint_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors", "control_v11p_sd15_lineart_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors", "control_v11p_sd15_mlsd_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors", "control_v11p_sd15_normalbae_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors", "control_v11p_sd15_openpose_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors", "control_v11p_sd15_scribble_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors", "control_v11p_sd15_seg_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors", "control_v11p_sd15_softedge_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors", "control_v11p_sd15s2_lineart_anime_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors", "control_v11f1e_sd15_tile_fp16.safetensors"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml", "control_v11e_sd15_ip2p_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml", "control_v11e_sd15_shuffle_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml", "control_v11p_sd15_canny_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml", "control_v11f1p_sd15_depth_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml", "control_v11p_sd15_inpaint_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml", "control_v11p_sd15_lineart_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml", "control_v11p_sd15_mlsd_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml", "control_v11p_sd15_normalbae_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml", "control_v11p_sd15_openpose_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml", "control_v11p_sd15_scribble_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml", "control_v11p_sd15_seg_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml", "control_v11p_sd15_softedge_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml", "control_v11p_sd15s2_lineart_anime_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml", "control_v11f1e_sd15_tile_fp16.yaml"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth", "t2iadapter_style_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth", "t2iadapter_sketch_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth", "t2iadapter_seg_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth", "t2iadapter_openpose_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth", "t2iadapter_keypose_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth", "t2iadapter_depth_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth", "t2iadapter_color_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth", "t2iadapter_canny_sd14v1.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth", "t2iadapter_canny_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth", "t2iadapter_depth_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth", "t2iadapter_sketch_sd15v2.pth"),
    ("https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth", "t2iadapter_zoedepth_sd15v1.pth")
]

upscale_files = [
    ("https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth", "RealESRGAN_x2plus.pth")
]

checkpoint_files = [
    ("https://huggingface.co/ckpt/Comic-Diffusion/resolve/main/comic-diffusion-V2.ckpt", "comic-diffusion-V2.ckpt")
]

for url, filename in controlnet_files:
    filepath = os.path.join(controlnet_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {controlnet_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

for url, filename in upscale_files:
    filepath = os.path.join(upscale_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {upscale_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

for url, filename in checkpoint_files:
    filepath = os.path.join(checkpoints_dir, filename)
    if not os.path.exists(filepath):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {checkpoints_dir} -o {filename}
    else:
        print(f"{filename} already exists, skipping download.")

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

!python main.py --dont-print-server